### Instalando bibliotecas

In [ ]:
pip install requests pandas pyarrow deep-translator

### Inicializacao

In [2]:
import requests
import pandas as pd
import os
import logging
from deep_translator import GoogleTranslator

# Instanciar o tradutor
translator = GoogleTranslator(source= "fr", target= "en")

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Função para salvar os dados brutos na pasta raw_parquet

In [3]:
def fetch_raw_data(name, raw_path, api_url):
    try:
        # Fazer a chamada na API
        response = requests.get(api_url)        
        # Verificar se a requisição foi bem-sucedida
        response.raise_for_status()  # Levanta um erro se o status não for 2xx
        # Verificar se o conteúdo é JSON
        if response.headers['Content-Type'].startswith('application/json'):
            data = response.json()            
            # Verificar se os dados retornados possuem a estrutura esperada
            if isinstance(data, list):
                df = pd.DataFrame(data)                
                # Verificar se o diretório de destino existe, caso contrário, criar
                os.makedirs(os.path.dirname(raw_path), exist_ok=True)                
                # Salvar os dados no formato Parquet
                df.to_parquet(raw_path, engine="pyarrow", index=False)
                logging.info(f"Arquivo '{raw_path}' criado/sobrescrito com sucesso.")
            else:
                logging.error(f"A resposta da API de {name} não contém uma lista de dados.")
        else:
            logging.error(f"A resposta da API de {name} não contém um JSON válido.")    
    except requests.exceptions.RequestException as e:
        # Erro na requisição (ex.: erro de rede, timeout, etc.)
        logging.error(f"Erro na chamada da API de {name}: {e}")    
    except Exception as e:
        # Outros erros (ex.: erro ao salvar o arquivo, erro ao processar os dados)
        logging.error(f"Ocorreu um erro ao processar os dados de {name}: {e}")

### Funções para salvar os dados filtrados na pasta filtered_parquet

In [4]:
def extract_nested_field(field, key, default=None):
    if isinstance(field, dict) and key in field:
        return field[key]
    return default

In [5]:
def filter_characters_data(raw_path, filtered_path):
    try:
        # Ler o arquivo Parquet
        df_characters = pd.read_parquet(raw_path, engine='pyarrow')
        # Verificar se o DataFrame tem dados
        if df_characters.empty:
            raise ValueError("O arquivo Parquet não contém dados.")
        # Criar um novo DataFrame com as colunas desejadas
        df_filtered = pd.DataFrame({
            'id': df_characters['id'],
            'name': df_characters['name'],
            'size': df_characters['size'],
            'age': df_characters['age'],
            'bounty': df_characters['bounty'],
            'crew_id': df_characters['crew'].apply(lambda x: extract_nested_field(x, 'id')),
            'fruit_id': df_characters['fruit'].apply(lambda x: extract_nested_field(x, 'id')),
            'job': df_characters['job'],
            'status': df_characters['status']
        })
        # Verificar e criar o diretório de saída se necessário
        os.makedirs(os.path.dirname(filtered_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_filtered.to_parquet(filtered_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{filtered_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")

In [6]:
def filter_fruits_data(raw_path, filtered_path):
    try:
        # Ler o arquivo Parquet
        df_fruits = pd.read_parquet(raw_path, engine='pyarrow')
        # Verificar se o DataFrame tem dados
        if df_fruits.empty:
            raise ValueError("O arquivo Parquet não contém dados.")
        # Criar um novo DataFrame com as colunas desejadas
        df_filtered = pd.DataFrame({
            'id': df_fruits['id'],
            'name': df_fruits['name'],
            'description': df_fruits['description'],
            'known_as': df_fruits['roman_name'],
            'type': df_fruits['type'],
            'fruit_image': df_fruits['filename']
        })
        # Verificar e criar o diretório de saída, se necessário
        os.makedirs(os.path.dirname(filtered_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_filtered.to_parquet(filtered_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{filtered_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")
    

In [7]:
def filter_crews_data(raw_path, filtered_path):
    try:
        # Ler o arquivo Parquet
        df_crews = pd.read_parquet(raw_path, engine='pyarrow')
        # Verificar se o DataFrame tem dados
        if df_crews.empty:
            raise ValueError("O arquivo Parquet não contém dados.")
        # Criar um novo DataFrame com as colunas desejadas
        df_filtered = pd.DataFrame({
            'id': df_crews['id'],
            'name': df_crews['name'],
            'status': df_crews['status'],
            'crewmate_number': df_crews['number'],
            'is_yonko': df_crews['is_yonko']
        })
        # Verificar e criar o diretório de saída, se necessário
        os.makedirs(os.path.dirname(filtered_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_filtered.to_parquet(filtered_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{filtered_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")

### Funções para tratar os dados de personagens

In [8]:
# Função para limpar e converter para int
def clean_and_convert_number(column):
    # Verifica se a coluna é do tipo string antes de aplicar .str
    if column.dtype == 'object':
        # Remove caracteres não numéricos
        column = column.str.replace(r'\D', '', regex=True)
    #  Substitui valores vazios por -1 e converte para inteiro
    return column.fillna(-1).replace("", -1).astype(int)

In [9]:
# Função para traduzir
def french_to_english(texto):
    if texto is None or pd.isna(texto):  # Verifica se o texto é nulo ou vazio
        return texto  # Retorna o valor original se for nulo
    try:
        return translator.translate(texto)
    except Exception as e:
        print(f"Erro ao traduzir '{texto}': {e}")
        return texto  # Retorna o texto original em caso de erro

In [15]:
def normalize_characters_data(filtered_path, normalized_path):
    try:
        df_normalized = pd.read_parquet(filtered_path, engine='pyarrow')
        # Colocar os valores das colunas em caixa baixa
        columns_to_lower = ["name", "job", "status"]
        df_normalized[columns_to_lower] = df_normalized[columns_to_lower].apply(lambda col: col.str.lower())
        # Aplicando a função de limpeza e conversão para as colunas desejadas
        columns_to_convert = ["size", "age", "bounty", "crew_id", "fruit_id"]
        df_normalized[columns_to_convert] = df_normalized[columns_to_convert].apply(clean_and_convert_number)
        # Substituir valores None ou em branco por "unknown" em todas as colunas
        columns_to_change = ["job", "status"]
        for column in columns_to_change:
            df_normalized[column] = df_normalized[column].replace("", None).fillna("unknown")

        # Verificar e criar o diretório de saída, se necessário
        os.makedirs(os.path.dirname(normalized_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_normalized.to_parquet(normalized_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{normalized_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")
        

In [16]:
def normalize_fruits_data(filtered_path, normalized_path):
    try:
        df_normalized = pd.read_parquet(filtered_path, engine='pyarrow')
        # Colocar os valores das colunas em caixa baixa
        columns_to_lower = ['name', 'description', 'known_as', 'type']
        df_normalized[columns_to_lower] = df_normalized[columns_to_lower].apply(lambda col: col.str.lower())
        # Substituir onde não possui imagem por "não disponível"
        df_normalized["fruit_image"] = df_normalized["fruit_image"].replace("https://images.api-onepiece.com/fruits/", "not available")
        # Manter apenas o texto antes da primeira vírgula
        df_normalized["known_as"] = df_normalized["known_as"].str.split(",", n=1).str[0]
        # Traduzir os textos em frances para ingles
        columns_to_translate = ['name', 'type']
        for column in columns_to_translate:
            df_normalized[column] = df_normalized[column].apply(french_to_english)

        # Verificar e criar o diretório de saída, se necessário
        os.makedirs(os.path.dirname(normalized_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_normalized.to_parquet(normalized_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{normalized_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")


In [17]:
def normalize_crews_data(filtered_path, normalized_path):
    try:
        df_normalized = pd.read_parquet(filtered_path, engine='pyarrow')
        # Colocar os valores das colunas em caixa baixa
        columns_to_lower = ['name', 'status']
        df_normalized[columns_to_lower] = df_normalized[columns_to_lower].apply(lambda col: col.str.lower())
        # Aplicando a função de limpeza e conversão para as colunas desejadas
        columns_to_convert = ["crewmate_number"]
        df_normalized[columns_to_convert] = df_normalized[columns_to_convert].apply(clean_and_convert_number)
        # Dicionário de substituições
        substituicoes = {
            "assets": "active",
            "dissolved": "dissolved",
            "unknown": "unknown",
            "inactive": "inactive",
            "arr�t�": "arrested",
            "actif": "active",
            "inconnu": "unknown",
            "inactif": "inactive",
            None: "unknown",
        }
        # Substituir os valores da coluna 'status'
        df_normalized["status"] = df_normalized["status"].replace(substituicoes)
        # Traduzir a coluna
        df_normalized["name"] = df_normalized["name"].apply(french_to_english)
    
        # Verificar e criar o diretório de saída, se necessário
        os.makedirs(os.path.dirname(normalized_path), exist_ok=True)
        # Salvar o DataFrame filtrado em Parquet
        df_normalized.to_parquet(normalized_path, engine="pyarrow", index=False)
        logging.info(f"Arquivo salvo com sucesso em '{normalized_path}'.")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")


### Função para processar os dados

In [18]:
def process_data(config_dados):
    # Percorrendo os valores de config_dados
    for config in config_dados:
        name = config['name']
        api_url = config.get('api_url')
        raw_path = config['raw_path']
        filtered_path = config['filtered_path']
        processing_filtered_function = config.get('filter_function')
        normalized_path = config['normalized_path']
        processing_normalize_function = config.get('normalize_function')

        logging.info(f"Processando {name}...")

        # Buscar os dados brutos
        try:
            fetch_raw_data(name, raw_path, api_url)
        except Exception as e:
            logging.error(f"Erro ao buscar dados brutos de {name}: {e}")
            continue

        if processing_filtered_function:
            try:
                processing_filtered_function(raw_path, filtered_path)
            except Exception as e:
                logging.error(f"Erro ao filtrar dados de {name}: {e}")
        else:
            logging.error(f"Nenhuma função definida para filtrar {name}.")

        if processing_normalize_function:
            try:
                processing_normalize_function(filtered_path, normalized_path)
            except Exception as e:
                logging.error(f"Erro ao normalizar dados de {name}: {e}")
        else:
            logging.error(f"Nenhuma função definida para normalizar {name}.")

        logging.info("-" * 40)

In [ ]:
config_dados = [
    {
        'name': 'personagens',
        'api_url': 'https://api.api-onepiece.com/v2/characters/en',
        'raw_path': 'raw_parquet/raw_characters.parquet',
        'filtered_path': 'filtered_parquet/filtered_characters.parquet',
        'filter_function': filter_characters_data,
        'normalized_path': 'normalized_parquet/normalized_characters.parquet',
        'normalize_function': normalize_characters_data
    },
    {
        'name': 'frutas',
        'api_url': 'https://api.api-onepiece.com/v2/fruits/en',
        'raw_path': 'raw_parquet/raw_fruits.parquet',
        'filtered_path': 'filtered_parquet/filtered_fruits.parquet',
        'filter_function': filter_fruits_data,
        'normalized_path': 'normalized_parquet/normalized_fruits.parquet',
        'normalize_function': normalize_fruits_data
    },
    {
        'name': 'tripulações',
        'api_url': 'https://api.api-onepiece.com/v2/crews/en',
        'raw_path': 'raw_parquet/raw_crews.parquet',
        'filtered_path': 'filtered_parquet/filtered_crews.parquet',
        'filter_function': filter_crews_data,
        'normalized_path': 'normalized_parquet/normalized_crews.parquet',
        'normalize_function': normalize_crews_data
    }
]

process_data(config_dados)